In [ ]:
#imports & froms

import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.image import imread
from PIL import Image
from keras.utils import to_categorical
from keras.models import Sequential,Model
from keras.layers import Dense,Conv2D,Flatten,Input,Activation,AveragePooling2D
from keras.models import model_from_json

In [ ]:
#data load

data =pd.read_csv("training1.csv")

In [ ]:
#data process

total_data=data.shape[0]
classes=10

nX=np.empty([total_data,64,64,3],dtype=np.float32)
nY=np.empty([total_data,1],dtype=np.int32)

for it in range(total_data):
    nY[it]=data['digit'][it]
    fname=data['filename'][it]
    im1=Image.open('trainingimage1/'+fname)
    im1=im1.resize((64,64),Image.NEAREST)
    my_image=np.array(im1)
    nX[it,:]=my_image

nY=to_categorical(nY)
indices=np.random.permutation(total_data)

idX_train,idX_test=indices[:int(total_data*0.8)],indices[int(total_data*0.8):]
trainX,testX=nX[idX_train,:],nX[idX_test,:]
trainY,testY=nY[idX_train,:],nY[idX_test,:]

trainX=trainX/255
testX=testX/255

In [ ]:
#model (Almost lenet-5)

X_input = Input((64,64,3))
X = Conv2D(6, (3, 3), strides=(1, 1),activation='relu', name='conv0')(X_input)
X = AveragePooling2D((2, 2))(X)

X = Conv2D(16, (3, 3), strides=(1, 1),activation='relu', name='conv1')(X_input)
X = AveragePooling2D((2, 2))(X)

X = Flatten()(X)
X = Dense(120, activation='relu', name='fc0')(X)
X = Dense(84, activation='relu', name='fc1')(X)

X = Dense(10, activation='softmax', name='fc2')(X)
model = Model(inputs=X_input, outputs=X, name='myModel')

In [ ]:
#run model

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(trainX,trainY,epochs=1,verbose=1)

In [ ]:
#saving model

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("modelWeights.h5")
print("Saved model to disk")

In [ ]:
#load model

json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("modelWeights.h5")

In [ ]:
#predict many
result=model.evaluate(testX,testY)
print(result[1])

#predicting Single Image
im1 = Image.open('ok6.png')
im2 = im1.resize((64, 64), Image.NEAREST) 
my_image = np.array(im2)
my_image=my_image/255
print(my_image.shape)
plt.imshow(my_image)
my_image=my_image.reshape(1,64,64,3)
print(model.predict(my_image))
